In [2]:
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm
import random
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# [1] 대구 행정구역별 카페 네이버 지도 크롤링

In [ ]:
# 주소

# 중구 : https://map.naver.com/p/search/%EB%8C%80%EA%B5%AC%20%EC%A4%91%EA%B5%AC%20%EC%B9%B4%ED%8E%98?c=13.00,0,0,0,dh
# 북구 : https://map.naver.com/p/search/%EB%8C%80%EA%B5%AC%20%EB%B6%81%EA%B5%AC%20%EC%B9%B4%ED%8E%98?c=11.00,0,0,0,dh
# 수성구 : https://map.naver.com/p/search/%EB%8C%80%EA%B5%AC%20%EC%88%98%EC%84%B1%EA%B5%AC%20%EC%B9%B4%ED%8E%98?c=12.00,0,0,0,dh
# 달서구 : https://map.naver.com/p/search/%EB%8C%80%EA%B5%AC%20%EB%8B%AC%EC%84%9C%EA%B5%AC%20%EC%B9%B4%ED%8E%98?c=12.00,0,0,0,dh
# 남구 : https://map.naver.com/p/search/%EB%8C%80%EA%B5%AC%20%EB%82%A8%EA%B5%AC%20%EC%B9%B4%ED%8E%98?c=13.00,0,0,0,dh
# 서구 : https://map.naver.com/p/search/%EB%8C%80%EA%B5%AC%20%EC%84%9C%EA%B5%AC%20%EC%B9%B4%ED%8E%98?c=13.00,0,0,0,dh
# 동구 : https://map.naver.com/p/search/%EB%8C%80%EA%B5%AC%20%EB%8F%99%EA%B5%AC%20%EC%B9%B4%ED%8E%98?c=11.00,0,0,0,dh
# 달성군 : https://map.naver.com/p/search/%EB%8C%80%EA%B5%AC%20%EB%8B%AC%EC%84%B1%EA%B5%B0%20%EC%B9%B4%ED%8E%98?c=10.00,0,0,0,dh

In [28]:
options = webdriver.ChromeOptions()

dic={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_5_2'}

service = Service(executable_path="./chromedriver.exe")
driver = webdriver.Chrome(service=service,options=options)
driver.get("https://map.naver.com/p/search/%EB%8C%80%EA%B5%AC%20%EB%8B%AC%EC%84%B1%EA%B5%B0%20%EC%B9%B4%ED%8E%98?c=10.00,0,0,0,dh")
driver.implicitly_wait(6)
time.sleep(3)

In [29]:
frame=driver.find_element(By.CSS_SELECTOR,"iframe#searchIframe")
driver.switch_to.frame(frame)

- 카페 이름 크롤링

In [30]:
# 페이지 넘기기
a=[]

click_=driver.find_elements(By.CLASS_NAME, 'eUTV2')

for n in tqdm(range(6)):
    time.sleep(1)

    # 스크롤 끝까지 내리기
    scroll = driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]')
    driver.execute_script("arguments[0].scrollBy(0,2000)", scroll)
    time.sleep(2)
    driver.execute_script("arguments[0].scrollBy(0,2000);", scroll)
    time.sleep(2)
    driver.execute_script("arguments[0].scrollBy(0,2000);", scroll)
    time.sleep(2)
    driver.execute_script("arguments[0].scrollBy(0,2000);", scroll)
    time.sleep(2)
    driver.execute_script("arguments[0].scrollBy(0,2000);", scroll)
    time.sleep(2)
    driver.execute_script("arguments[0].scrollBy(0,2000);", scroll)
    time.sleep(2)
    driver.execute_script("arguments[0].scrollBy(0,2000);", scroll)
    time.sleep(2)
        
    try:
        # 카페 이름 따기
        main=driver.find_elements(By.CSS_SELECTOR, "div > span.place_bluelink.TYaxT")
        print(len(main))

        for i in main:
            a.append(i.text)
            
    except Exception as ea:
        break
    click_[1].click()

  0%|          | 0/6 [00:00<?, ?it/s]

54


 17%|█▋        | 1/6 [00:18<01:31, 18.39s/it]

50


 33%|███▎      | 2/6 [00:34<01:07, 16.78s/it]

...

 67%|██████▋   | 4/6 [01:05<00:32, 16.18s/it]

50


 83%|████████▎ | 5/6 [01:21<00:15, 15.97s/it]

50


100%|██████████| 6/6 [01:37<00:00, 16.22s/it]


In [32]:
df=pd.DataFrame(a)
df.columns=['name']
df.to_excel('달성군카페.xlsx',encoding='utf-8')


c:\Users\KDP-48\anaconda3\envs\MY_PYTHON2\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [ ]:
df=pd.read_excel('./달성군카페.xlsx',index_col=False)
df

In [34]:
b=[]
for i in df['name']:
    i=i.replace(' ','')
    b.append(i)
df['name']=b

# 카페 이름을 네이버 통합검색에 검색해서 크롤링

In [35]:
a=[]

for i in tqdm(df['name']):
    url=f"https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=대구+{i}"
    response=requests.get(url,headers={'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 13_5_2)"})
    bs=BeautifulSoup(response.text,'html.parser')

    time.sleep(1.5+random.random())

    try:
        name = bs.find('span',class_='Fc1rA').text
        local = bs.find('span',class_='LDgIH').text
        review = bs.find('a',class_='place_bluelink').text
        number = bs.find('span',class_ ='xlx7Q').text
        a.append([name,local,review,number,url])
    except:
        pass

100%|██████████| 304/304 [13:02<00:00,  2.57s/it]


In [36]:
DF=pd.DataFrame(a)
DF.columns=['가게이름','주소','방문자리뷰','전화번호','url']
DF.to_excel('달성군카페DF.xlsx',encoding='utf-8')

c:\Users\KDP-48\anaconda3\envs\MY_PYTHON2\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


# 카페 위치의 위도, 경도 구하기

In [15]:
# 위도만
def lat(address):
    from geopy.geocoders import Nominatim
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    lat = geo.latitude
    return lat

# 경도만
def lng(address):
    from geopy.geocoders import Nominatim
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    lng = geo.longitude
    return lng

# 위도 경도 둘다
def geocoding(address):
    from geopy.geocoders import Nominatim
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    crd = {"lat": geo.latitude, "lng": geo.longitude}
    return crd

In [45]:
DF1=pd.read_excel('./수성구카페DF.xlsx')
DF2=pd.read_excel('./서구카페DF.xlsx')
DF3=pd.read_excel('./북구카페DF.xlsx')
DF4=pd.read_excel('./동구카페DF.xlsx')
DF5=pd.read_excel('./달서구카페DF.xlsx')
DF6=pd.read_excel('./남구카페DF.xlsx')
DF7=pd.read_excel('./달성군카페DF.xlsx')
DF8=pd.read_excel('./중구카페DF.xlsx')

In [39]:
DF=pd.concat([DF1,DF2,DF3,DF4,DF5,DF6,DF7])

In [12]:
Food=pd.read_excel('./DaeguFood.xlsx')
Food=Food[['가게명','주소','리뷰갯수','전화번호','지역']]

In [16]:
a=Food['주소'].to_list()
b=[]
c=[]

for i in tqdm(a) :
    try:
        b.append(lat(i))
        c.append(lng(i))
    except:
        b.append(0)
        c.append(0)

100%|██████████| 1763/1763 [34:16<00:00,  1.17s/it]


In [18]:
Food['위도']=b
Food['경도']=c

In [20]:
Food.to_csv('daeguFood.csv',encoding='utf-8')

In [49]:
DF_f=DF[['가게이름','주소','방문자리뷰','전화번호','url','위도','경도']]

In [6]:
ab=[]

for i in Food['리뷰갯수']:
    i=i.replace(',','')
    if i=='전체보기':
        pass
    else:
        ab.append(i)

Food['리뷰갯수']=ab

In [16]:
DF_f

,가게이름,주소,방문자리뷰,전화번호,url,위도,경도
0,카페오굿,대구 수성구 달구벌대로492길 45-2,200,0507-0289-3526,https://search.naver.com/search.naver?where=ne...,35.857867,128.632206
1,카페동이 2호점,대구 수성구 수성로69길 65,2243,0507-1313-0614,https://search.naver.com/search.naver?where=ne...,35.856049,128.610679
2,무화커피,대구 수성구 국채보상로186길 81,1271,0507-1437-9188,https://search.naver.com/search.naver?where=ne...,35.863602,128.634176
3,딥커피로스터스 범어,대구 수성구 범어천로 155-1,32,0507-1487-3444,https://search.naver.com/search.naver?where=ne...,35.863935,128.618797
4,에폰베이크샵,대구 수성구 범어로24길 14,116,0507-1431-5153,https://search.naver.com/search.naver?where=ne...,35.860216,128.636523
...,...,...,...,...,...,...,...
1869,삼더크,대구 중구 달구벌대로443길 44-3,179,0507-1376-1010,https://search.naver.com/search.naver?where=ne...,35.864579,128.606755
1870,유어박스 동성로점,대구 중구 중앙대로81길 13,43,0507-1378-9122,https://search.naver.com/search.naver?where=ne...,35.869509,128.589466
1871,키오키,대구 중구 달구벌대로443길 7-19,165,0507-1305-4107,https://search.naver.com/search.naver?where=ne...,35.863430,128.604286
1872,동그라미과자점,대구 중구 명륜로23길 26,255,0507-1446-5010,https://search.naver.com/search.naver?where=ne...,35.861698,128.596045


In [18]:
DF_f.to_excel('카페DF.xlsx',encoding='utf-8')

# MariaDB에 데이터추가

In [2]:
import mariadb as mdb

conn_params = {'host':'172.20.41.42',
               'user':'member2',
               'passwd' : 'member2',
               'port' : 3307,
               'db' : 'ProjectDB',
               'autocommit' : True}

In [24]:
try:

    # mariaDB 연결
    connDB=mdb.connect(**conn_params)

    # DB에 접근할 수 있는 Cursor객체 가져오기
    cursor=connDB.cursor()

    # # DB에 테이블 만들기
    create_table_query = """
    CREATE TABLE Cafe (
        가게이름 varchar(100),
        주소 varchar(255),
        방문자리뷰 ,
        전화번호 varchar(30),
        url varchar(255),
        위도 float,
        경도 float
    )
    """
    cursor.execute(create_table_query)

    
    # 엑셀 파일 경로
    excel_path='./카페DF.xlsx'

    # 엑셀 파일 읽기
    data_frame = pd.read_excel(excel_path)

    # 데이터 프레임 반복문으로 순회하며 INSERT 쿼리 실행
    for index, row in data_frame.iterrows():
        column1_value = row['가게이름']
        column2_value = row['주소']
        column3_value = int(row['방문자리뷰'])
        column4_value = row['전화번호']
        column5_value = row['url']
        column6_value = row['위도']
        column7_value = row['경도']

        # INSERT 쿼리 실행
        insert_query = f"INSERT INTO Cafe (가게이름, 주소, 방문자리뷰, 전화번호, url, 위도, 경도)
        VALUES ('{column1_value}', '{column2_value}','{column3_value}','{column4_value}',
        '{column5_value}','{column6_value}','{column7_value}')"
        cursor.execute(insert_query)


    print('OK')
    # 현재 데이터베이스의 모든 테이블 조회 쿼리 실행
    show_tables_query = "SHOW TABLES"
    cursor.execute(show_tables_query)

    # 결과 가져오기
    tables = cursor.fetchall()
    print('OK')

    # 변경사항 저장 및 연결 종료
    connDB.commit()
    connDB.close()





except mdb.Error as e:
    print(f'Error: {e}')